In [6]:
from tensorflow.keras.models import load_model, Sequential
import numpy as np

# ✅ Load your model
model = load_model("cnn model\cnnFace.keras")  # or .keras

# ✅ Force model to build by calling it once
dummy_input = np.zeros((1, 128, 128, 1), dtype=np.float32)  # adjust size if different
_ = model.predict(dummy_input)

# ✅ Now safely remove the final layer
embedding_model = Sequential(model.layers[:-1])

# ✅ Test the embedding model
print("✅ Embedding model created successfully!")
print("Output shape:", embedding_model.predict(dummy_input).shape)



<>:5: SyntaxWarning: invalid escape sequence '\c'
<>:5: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Harsh\AppData\Local\Temp\ipykernel_23428\783434265.py:5: SyntaxWarning: invalid escape sequence '\c'
  model = load_model("cnn model\cnnFace.keras")  # or .keras


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
✅ Embedding model created successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step
Output shape: (1, 128)


In [ ]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern

# ✅ CLAHE Function
def apply_CLAHE(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(image)

# ✅ Retinex Function (Simple Single-Scale)
def apply_retinex(image):
    image = image.astype(np.float32) + 1.0  # Prevent log(0)
    retinex = np.log(image) - np.log(cv2.GaussianBlur(image, (0, 0), sigmaX=30))
    retinex = cv2.normalize(retinex, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    return retinex.astype(np.uint8)

# ✅ Uniform LBP Function
def apply_uniform_lbp(image, P=8, R=1):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    lbp = local_binary_pattern(image, P, R, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(),
                             bins=np.arange(0, P + 3),
                             range=(0, P + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)  # Normalize
    return hist

# ✅ Final Embedding Function
def get_embedding(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Image not found: {img_path}")

    # Step 1: Resize (optional first resize if images are inconsistent)
    img = cv2.resize(img, (128, 128))

    # Step 2: Apply CLAHE
    img = apply_CLAHE(img)

    # Step 3: Apply Retinex
    img = apply_retinex(img)

    # Step 4: Apply Uniform LBP
    img = apply_uniform_lbp(img)
    print(img)


  


In [9]:
# ✅ Test get_embedding() on one image
test_img_path = "img.jpg"  # update to an existing file

try:
    emb = get_embedding(test_img_path)
    print("✅ Embedding generated successfully!")
    print("Embedding shape:", emb.shape)
    print("Embedding sample values:", emb[:10])
except Exception as e:
    print("❌ Error:", e)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
✅ Embedding generated successfully!
Embedding shape: (128,)
Embedding sample values: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [10]:
embedding_model = Sequential(model.layers[:-1])


In [11]:
for idx, layer in enumerate(model.layers):
    try:
        print(idx, layer.name, layer.output_shape)
    except:
        print(idx, layer.name, "⚠️ No output shape (maybe not built yet)")


0 conv2d ⚠️ No output shape (maybe not built yet)
1 batch_normalization ⚠️ No output shape (maybe not built yet)
2 max_pooling2d ⚠️ No output shape (maybe not built yet)
3 conv2d_1 ⚠️ No output shape (maybe not built yet)
4 batch_normalization_1 ⚠️ No output shape (maybe not built yet)
5 max_pooling2d_1 ⚠️ No output shape (maybe not built yet)
6 conv2d_2 ⚠️ No output shape (maybe not built yet)
7 batch_normalization_2 ⚠️ No output shape (maybe not built yet)
8 max_pooling2d_2 ⚠️ No output shape (maybe not built yet)
9 flatten ⚠️ No output shape (maybe not built yet)
10 dense ⚠️ No output shape (maybe not built yet)
11 dropout ⚠️ No output shape (maybe not built yet)
12 dense_1 ⚠️ No output shape (maybe not built yet)


In [14]:
from tensorflow.keras.models import load_model, Sequential
import numpy as np

# ✅ 1. Load the original model
model = load_model("cnn model/cnnFace.keras")  # Update path if needed

# ✅ 2. Force the model to build
dummy_input = np.zeros((1, 128, 128, 1), dtype=np.float32)
_ = model.predict(dummy_input)  # Builds the model graph

# ✅ 3. Create embedding model by removing last TWO layers
# Final layers:
# 10 = Dense
# 11 = Dropout
# 12 = Dense (classifier)
embedding_model = Sequential(model.layers[:11])  # keep up to index 10

# ✅ 4. Test output
print("✅ Embedding model created using layer slicing!")
test_emb = embedding_model.predict(dummy_input)
print("Output shape:", test_emb.shape)
print("Sample values:", test_emb[0][:10])


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 633ms/step
✅ Embedding model created using layer slicing!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step
Output shape: (1, 128)
Sample values: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


directly processing the classfication cnn 

In [15]:
from tensorflow.keras.models import load_model
model = load_model("cnn model/cnnFace.keras")  # update path if needed


In [ ]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern

# ✅ CLAHE
def apply_CLAHE(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(image)

# ✅ Retinex
def apply_retinex(image):
    image = image.astype(np.float32) + 1.0
    retinex = np.log(image) - np.log(cv2.GaussianBlur(image, (0, 0), sigmaX=30))
    retinex = cv2.normalize(retinex, None, 0, 255, cv2.NORM_MINMAX)
    return retinex.astype(np.uint8)

# ✅ LBP
def apply_uniform_lbp(image, P=8, R=1):
    lbp = local_binary_pattern(image, P, R, method="uniform")
    lbp = (lbp / lbp.max()) * 255
    return lbp.astype(np.uint8)

# ✅ Predict function
def predict_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (128, 128))

    img = apply_CLAHE(img)
    img = apply_retinex(img)
    img = apply_uniform_lbp(img)

    img = img / 255.0
    img = img.reshape(1, 128, 128, 1)

    prediction = model.predict(img)
    class_id = np.argmax(prediction)
    confidence = np.max(prediction)

    return class_id, confidence


In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hearfool/vggface2")

print("Path to dataset files:", path)

Resuming download from 991952896 bytes (1502058198 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/hearfool/vggface2?dataset_version_number=1 (991952896/2494011094) bytes left.


100%|██████████| 2.32G/2.32G [08:08<00:00, 3.07MB/s]

Extracting files...


Path to dataset files: C:\Users\Harsh\.cache\kagglehub\datasets\hearfool\vggface2\versions\1
